In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.preprocessing import LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import f1_score

In [2]:
def find_fibra_quota(x):
    if not x.find("FIBRA")==-1:
        ix = x.find("FIBRA")+6
        iy = x[ix:].find(" ")
        iz = x[(ix+iy+1):].find(" ")
        whatwehave = x[ix: (ix+iy+iz+1)]
        if whatwehave.find("GB")!= -1:
            return float(whatwehave[:whatwehave.find(" ")])*1000
        else:
            return float(whatwehave[:whatwehave.find(" ")])
    return 100

### Read Data

In [4]:
main = pd.read_csv("train.csv")
main_test = pd.read_csv("test.csv")
main = pd.concat([main,main_test],axis=0)

FileNotFoundError: [Errno 2] File b'train.csv' does not exist: b'train.csv'

In [3]:
main = pd.read_csv("./data/model_data.csv")

### Define Columns

In [6]:
target = "Churn.30"
relevant_columns = main.columns.tolist()
relevant_columns.remove(target)

### Exploring the columns

In [7]:
for col in relevant_columns:
    print("="*5,end=" ")
    print(col,end=" ")
    print("="*5)
    print(main[col].value_counts(dropna=False))

===== ultimo_ivr_departamento =====
ATC                 41676
SOPORTE A VENTAS     1330
VENTAS                697
COBROS                 60
Missing                58
CANCELACIONES          50
ATC NEGOCIO             1
Name: ultimo_ivr_departamento, dtype: int64
===== ultima_categoria_open_question =====
AVERIAS                  5810
Missing                  4876
INTERNET                 4291
TARIFAS                  3562
TERMINAL                 2941
FACTURA                  2773
AGENTE                   2765
RECLAMACION              1927
SUSCRIPCION              1879
PROMOCIONES              1792
SIM                      1261
BAJA                      979
PEDIDO                    871
CONTRATO                  836
PORTABILIDAD              760
TV                        685
ALTA                      658
COBERTURA                 639
ROAMING                   629
INSTALACION               599
DEUDA                     595
RENUEVO                   577
PREMIUM                   451
GENER

### Correct type of the columns

In [8]:
main["grupo_acd"] = main["grupo_acd"].astype(str)

### Create New Features

In [9]:
main["client_segment"] = main.grupo_acd.apply(lambda x: x[-3:])
main["bpo"] = main.grupo_acd.apply(lambda x: x[:2])

In [10]:
main["tariff_ds_has_fibra"] = main.tariff_ds.apply(lambda x: x.find("FIBRA")!=-1)
main["tariff_ds_fibra_quota"] = main.tariff_ds.apply(find_fibra_quota)

### Export Feature Dictionary & Final Data

In [12]:
dtypes_ = dict(zip(main.dtypes.index,main.dtypes))
dtypes_.pop(target)
# dtypes_.pop('TARGET')
# dtypes_.pop('id')

dtype('int64')

In [13]:
dtypes_df = pd.DataFrame.from_dict(dtypes_,orient="index",columns=["dtype"])
dtypes_df.reset_index(inplace=True)
dtypes_df.rename(columns={"index":"feature"},inplace=True)
dtypes_df["dtype2"] = dtypes_df.dtype.apply(lambda x: "cont" if x in [np.dtype("int64"),np.dtype("float64")] else "cat")
dtypes_df["dtype2"] = dtypes_df.apply(lambda x: "cat" if (x.feature.find("_transformed")!=-1 or x.feature.find("_clustered")!=-1) else x.dtype2, axis=1)

In [14]:
dtypes_df["distinct_count"] = dtypes_df.feature.apply(lambda x: main[x].value_counts(dropna=False).shape[0]) 
dtypes_df["na_count"] = dtypes_df.feature.apply(lambda x: main[main[x].isna()].shape[0])
dtypes_df["na_perc"] = dtypes_df.na_count.apply(lambda x: x/main.shape[0])

In [15]:
dtypes_df.to_csv('feature_dictionary.csv')

In [ ]:
main.to_csv('./data/model_data.csv',index=False)